In [65]:
import os
import csv
import math
from urllib.request import urlopen, Request

import OpenDartReader
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from pykrx import stock
import time

import cjw_maria
from cjw_main import *
import numpy as np


In [66]:
cols = ['날짜', '자산', '부채', '자본', '기초현금및현금성자산', '기말현금및현금성자산', 
       '유동자산계산에 참여한 계정 펼치기', '유동부채계산에 참여한 계정 펼치기', 
        '매출액', '영업이익', '영업이익(발표기준)', '당기순이익']
changed_cols = [c.replace('계산에 참여한 계정 펼치기', '') for c in cols]
changed_cols = [c.replace('(', '') for c in changed_cols]
changed_cols = [c.replace(')', '') for c in changed_cols]

def getFN(code):
    try:
        df = pd.read_html(get_html_fnguide(code, 0))
        df = pd.concat([df[0][df[0].columns[:5]], df[2], df[4]])
        df = df.transpose().reset_index(drop=False)
        df.columns = df.iloc[0]
        df = df.drop(0).reset_index(drop=True)
        fs = df.columns[0] # 연결 or 개별?
        df = df.rename(columns={df.columns[0]: "날짜"})
        df = df[cols]
        df['FS'] = [fs] * df.shape[0]
        df.columns = changed_cols + ['FS']
        return df
    except:
        writeError('error3', code, 'A')
        return

maria = cjw_maria.MariaDB()

In [67]:
stock_amount_2016 = stock.get_market_cap_by_ticker("20161229 ")[['상장주식수']]
code_list = list(stock_amount_2016.index)

for batch in range(0, 2240, 10):
    df_2017 = []
    df_2018 = []
    df_2019 = []
    df_2020 = []
    print('batch: {}'.format(batch))
    for code in code_list[batch:batch+10]:
        print(code)
        try:
            df = getFN(code)
            df_2017.append([code] + list(df.iloc[0]))
            df_2018.append([code] + list(df.iloc[1]))
            df_2019.append([code] + list(df.iloc[2]))
            df_2020.append([code] + list(df.iloc[3]))
        except:
            writeError('error3', code, 'B')
            continue

    df_2017 = pd.DataFrame(df_2017)
    df_2018 = pd.DataFrame(df_2018)
    df_2019 = pd.DataFrame(df_2019)
    df_2020 = pd.DataFrame(df_2020)

    df_2017.columns = ['티커'] + changed_cols + ['FS']
    df_2018.columns = ['티커'] + changed_cols + ['FS']
    df_2019.columns = ['티커'] + changed_cols + ['FS']
    df_2020.columns = ['티커'] + changed_cols + ['FS']

    df_2017 = df_2017.set_index('티커')[changed_cols[1:] + ['FS']]
    df_2018 = df_2018.set_index('티커')[changed_cols[1:] + ['FS']]
    df_2019 = df_2019.set_index('티커')[changed_cols[1:] + ['FS']]
    df_2020 = df_2020.set_index('티커')[changed_cols[1:] + ['FS']]

    stock_amount_2017 = stock.get_market_cap_by_ticker("20171228 ")[['상장주식수']]
    stock_amount_2018 = stock.get_market_cap_by_ticker("20181228 ")[['상장주식수']]
    stock_amount_2019 = stock.get_market_cap_by_ticker("20191230 ")[['상장주식수']]
    stock_amount_2020 = stock.get_market_cap_by_ticker("20201230 ")[['상장주식수']]

    df_2017 = pd.merge(df_2017, stock_amount_2017, left_index=True, right_index=True)
    df_2018 = pd.merge(df_2018, stock_amount_2018, left_index=True, right_index=True)
    df_2019 = pd.merge(df_2019, stock_amount_2019, left_index=True, right_index=True)
    df_2020 = pd.merge(df_2020, stock_amount_2020, left_index=True, right_index=True)
    
    df_2017 = df_2017[changed_cols[1:] + ['상장주식수', 'FS']].fillna(-9999999)
    df_2018 = df_2018[changed_cols[1:] + ['상장주식수', 'FS']].fillna(-9999999)
    df_2019 = df_2019[changed_cols[1:] + ['상장주식수', 'FS']].fillna(-9999999)
    df_2020 = df_2020[changed_cols[1:] + ['상장주식수', 'FS']].fillna(-9999999)
    
    print('start save!')
    for i in range(df_2017.shape[0]):
        maria.insertData('f2017', tuple([df_2017.index[i]]+list(map(int, list(df_2017.iloc[i][df_2017.columns[:-1]])))+[df_2017['FS'].iloc[i][-3:-1]]))
        maria.commitDB()
    print('finish f2017!')
    for i in range(df_2018.shape[0]):
        maria.insertData('f2018', tuple([df_2018.index[i]]+list(map(int, list(df_2018.iloc[i][df_2018.columns[:-1]])))+[df_2018['FS'].iloc[i][-3:-1]]))
        maria.commitDB()
    print('finish f2018!')
    for i in range(df_2019.shape[0]):
        maria.insertData('f2019', tuple([df_2019.index[i]]+list(map(int, list(df_2019.iloc[i][df_2019.columns[:-1]])))+[df_2019['FS'].iloc[i][-3:-1]]))
        maria.commitDB()
    print('finish f2019!')
    for i in range(df_2020.shape[0]):
        maria.insertData('f2020', tuple([df_2020.index[i]]+list(map(int, list(df_2020.iloc[i][df_2020.columns[:-1]])))+[df_2020['FS'].iloc[i][-3:-1]]))
        maria.commitDB()
    print('finish f2020!')

batch: 0
005930
000660
005380
005935
015760
012330
035420
028260
032830
005490
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 10
055550
090430
017670
105560
051910
034730
000270
033780
096770
051900
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 20
000810
011170
068270
034220
009540
002790
018260
003550
207940
010950
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 30
086790
010130
000030
066570
030200
035250
004020
006400
161390
024110
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 40
023530
021240
047810
086280
088350
018880
001040
036570
035720
004800
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 50
139480
078930
032640
000720
097950
009240
029780
036460
005830
000120
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 60
009830
027410
001800
002380
009150
005387
007070
010140
241560
008930
start save!
finish f2017!
finish 

005950
095660
200230
001460
900260
002900
039570
011040
087010
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 560
060300
005680
067000
183190
000680
000540
014470
007390
004250
018120
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 570
001200
000860
060250
074600
084010
036030
004060
004970
900270
086520
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 580
011370
950140
078070
114120
097955
035150
004840
082740
054050
005160
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 590
064260
136490
137940
000020
004380
000970
001940
002030
010690
011160
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 600
005389
010955
084680
035890
011280
015360
060150
193250
011930
000180
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 610
195870
023160
019210
109740
052330
194370
013520
061460
234080
067830
start save!
finish f2017!
finish f2018!
fin

004780
060370
003960
012790
030350
020180
008040
050890
007110
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1110
024890
053610
900070
056700
143160
005010
122350
170920
008250
214270
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1120
091970
057540
050090
058850
036000
083930
149980
067990
048870
106190
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1130
059210
045970
155650
205100
019770
142280
215200
128820
071670
071280
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1140
027740
040300
127710
014990
053620
049430
145270
009780
149950
049470
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1150
079960
206560
058630
005190
210540
065420
054040
053280
009270
189980
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1160
006980
027580
011500
002230
081000
208140
004830
123330
216400
014160
start save!
finish f2017!
finish f201

finish f2019!
finish f2020!
batch: 1650
001550
043910
075970
053110
073490
200470
177830
000760
032820
005420
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1660
126880
182360
086830
104460
052460
003720
010770
220100
150840
097800
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1670
065690
037400
093520
003010
065500
115570
013000
028040
045890
109820
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1680
002680
004410
170790
099220
094970
057680
011300
093230
001470
079000
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1690
036260
007770
109070
024060
035460
090150
067730
078940
083450
051390
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1700
002220
250300
054920
131400
119500
006920
203450
213090
124500
003465
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 1710
189690
033310
005820
217820
071950
009190
007280
039670
1509

000145
179720
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 2200
016385
060910
000725
185190
189540
247300
099350
00781K
005965
090355
start save!
finish f2017!
finish f2018!
finish f2019!
finish f2020!
batch: 2210
010145
001065
004835
001527
001755
003415
004985
004415
003075
011155


ValueError: Length mismatch: Expected axis has 0 elements, new values have 14 elements

In [63]:
df_2017

""


In [64]:
len(code_list)

2249